In [1]:
%%HTML
<script src="require.js"></script>

In [ ]:
# Import necessary modules

<img src='https://i.imgur.com/jzRQsuC.png' width = 600>

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">Abstract</h1>
</div>

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">Background</h1>
</div>

<div style="background-color: #fff3e4; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h2 style="color: #5a3731; margin: 0;">Problem Statement</h2>
</div>

The study aims to answer the main problem: **How can explicit user ratings be utilized to recommend board games based on similar users?**

To address the main question, the study will also uncover answers to the following subquestion:
- Which collaborative filtering methods, neighborhood- or model-based method, result in a better performance?


<div style="background-color: #fff3e4; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h2 style="color: #5a3731; margin: 0;">Motivation</h2>
</div>

<div style="background-color: #fff3e4; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h2 style="color: #5a3731; margin: 0;">Key Terms</h2>
</div>

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">Methodology</h1>
</div>

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">Data Collection</h1>
</div>

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">Data Pre-Processing and Cleaning</h1>
</div>

In [ ]:
orginal_df = pd.read_csv("/mnt/data/public/bgg/bgg-19m-reviews.csv")

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">Exploratory Data Analysis</h1>
</div>

<div style="background-color: #ffd3a0; width: 200px; padding: 10px; border-radius: 5px; width: 98.8%">
    <h1 style="color: #5a3731; margin: 0;">References</h1>
</div>

Van Elteren, J. (2022). *BoardGameGeek Reviews.* https://www.kaggle.com/datasets/jvanelteren/boardgamegeek-reviews/data